## LayerNorm (층 정규화)

1. **평균($\mu$)과 분산($\sigma^2$) 구하기**
   - 입력 벡터($x$)의 차원($d$) 내에서 통계량을 계산
   $$\mu = \frac{1}{d} \sum_{i=1}^{d} x_i$$
   $$\sigma^2 = \frac{1}{d} \sum_{i=1}^{d} (x_i - \mu)^2$$

2. **정규화 (Normalization)**
   - 평균이 0, 분산이 1이 되도록 값을 조정 ($\epsilon$은 분모가 0이 되는 것을 막기 위한 작은 수)
   $$\hat{x}_i = \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}}$$

3. **스케일링 & 이동 (Scale & Shift)**
   - 단순히 정규화만 하면 데이터가 가진 고유한 표현력이 사라질 수 있음
   - 학습 가능한 파라미터인 감마($\gamma$)와 베타($\beta$)를 도입해서, 모델이 알아서 적절한 범위로 조절
   $$y_i = \gamma \cdot \hat{x}_i + \beta$$

In [ ]:
import os
import torch

In [6]:
x = torch.tensor([1.0, 10.0, 100.0])
print(x)

mean = x.mean()
var = x.var(unbiased=False) # 모분산
std = torch.sqrt(var + 1e-6)

print(mean)
print(var)
print(std)

x_norm = (x - mean) / std
print(x_norm)

tensor([  1.,  10., 100.])
tensor(37.)
tensor(1998.)
tensor(44.6990)
tensor([-0.8054, -0.6040,  1.4094])


In [23]:
# L, dim
L, dim = 10, 256
x = torch.randn(L, dim)
print(x.shape)

mean = torch.mean(x, dim=-1, keepdim=True)
var = torch.var(x, dim=-1, unbiased=False ,keepdim=True)
std = torch.sqrt(var + 1e-6)

x_norm = (x-mean) / std
print(x_norm.shape)

gamma = torch.ones(dim)
beta = torch.zeros(dim)

y = x_norm * gamma + beta

print(y.shape)

torch.Size([10, 256])
torch.Size([10, 256])
torch.Size([10, 256])


In [ ]:
# Layer Normalization
# B, L, dim
B, L, dim = 4, 10, 512
x = torch.randn(B, L, dim)
print(x.shape)

mean = torch.mean(x, dim=-1, keepdim=True)
var = torch.var(x, dim=-1, unbiased=False ,keepdim=True)
std = torch.sqrt(var + 1e-6)

x_norm = (x-mean) / std
print(x_norm.shape)

gamma = torch.ones(dim)
beta = torch.zeros(dim)

y = x_norm * gamma + beta

print(y.shape)


torch.Size([4, 10, 512])
torch.Size([4, 10, 512])
torch.Size([4, 10, 512])


## Batch Normalization
1. 평균($\mu_{\mathcal{B}}$)과 분산($\sigma_{\mathcal{B}}^2$) 구하기
    - 배치($m$) 전체를 보고, 같은 위치(Feature)에 있는 값들의 통계량을 계산
    - $m$: 배치 크기 (Batch Size)
    $$\mu_{\mathcal{B}} = \frac{1}{m} \sum_{i=1}^{m} x_i$$
    $$\sigma_{\mathcal{B}}^2 = \frac{1}{m} \sum_{i=1}^{m} (x_i - \mu_{\mathcal{B}})^2$$
1. 정규화 (Normalization)
    - 배치 내에서 평균이 0, 분산이 1이 되도록 값을 조정 ($\epsilon$은 안정성을 위한 작은 수)
    $$\hat{x}_i = \frac{x_i - \mu_{\mathcal{B}}}{\sqrt{\sigma_{\mathcal{B}}^2 + \epsilon}}
1. $$스케일링 & 이동 (Scale & Shift)
    - 정규화된 값이 데이터의 고유한 특징을 잃지 않도록 조정
    - 학습 가능한 파라미터 감마($\gamma$)와 베타($\beta$) 사용 (채널/특성별로 존재)
    $$y_i = \gamma \cdot \hat{x}_i + \beta$$

Note: 학습(Train) 때는 위 계산을 매번 수행하지만, 추론(Inference/Eval) 때는 학습 중 구해놓은 '이동 평균(Moving Average)'을 사용한다는 점이 LayerNorm과 가장 큰 차이입니다.

In [26]:
# Batch Normalization
# B, L, dim
B, L, dim = 4, 10, 512
x = torch.randn(B, L, dim)
print(x.shape)

mean_bn = torch.mean(x, dim=0, keepdim=True)
var_bn = torch.var(x, dim=0, unbiased=False ,keepdim=True)
std_bn = torch.sqrt(var_bn + 1e-6)

x_norm = (x-mean_bn) / std_bn
print(x_norm.shape)

gamma = torch.ones(dim)
beta = torch.zeros(dim)

y = x_norm * gamma + beta

print(y.shape)


torch.Size([4, 10, 512])
torch.Size([4, 10, 512])
torch.Size([4, 10, 512])
